# QA of DSL Release v1.26
The purpose of this notebook is to review and test the new features in this DSL version.

Docs: https://docs.dimensions.ai/dsl/1.26.0-preview/


## Prerequisites 

Please install the latest versions of these libraries to run this notebook. 

In [47]:
#
# load common libraries
import pandas as pd
from pandas.io.json import json_normalize

import time
import json
from tqdm.notebook import tqdm as progress

import plotly.express as px
from plotly.offline import plot 

import dimcli
from dimcli.shortcuts import *

dimcli.login(instance="test")
dsl = dimcli.Dsl() 


Dimcli - Dimensions API Client (v0.7.1.1)
Connected to: https://integration.ds-metrics.com - DSL v1.26
Method: dsl.ini file


---

## [DSL-458] Expert search 500 errors

https://uberresearch.atlassian.net/browse/DSL-458

In [48]:
%%dsl 

identify experts 
    from concepts ["ambipolar electric field effect", "two-dimensional semimetal", "room-temperature mobility", "electric field effects", "field effects", "graphitic films", "gate voltage", "conductance band", "square centimeter", "films"]
    where research_org_countries is not empty
    and year >= 2013
return experts
annotate coauthorship, organizational overlap
with ["ur.012332544274.51", "ur.012364005435.27"]

<dimcli.DslDataset object #4805625104. Dict keys: '_stats', '_version', 'experts'>

## [DSL-263] Faceting on grants researchers

https://uberresearch.atlassian.net/browse/DSL-263

In [49]:
%%dsldf

search grants where researchers.id in ["ur.0723426172.10", "ur.01277776417.51"] 
return researchers[id+total_grants+last_grant_year]

Returned Researchers: 13


id  count  last_grant_year  total_grants
0     ur.0723426172.10      8             2018             8
1   ur.012662217132.90      2             2018            13
2   ur.010520247252.54      1             2003             2
3    ur.01121653260.31      1             2022            18
4    ur.01127672147.84      1             2020            14
5   ur.011316554452.18      1             1998             1
6   ur.012167132327.75      1             2025            11
7   ur.012237141052.77      1             1996             1
8   ur.012455520474.57      1             2003             6
9    ur.01274135317.46      1             2018             4
10  ur.013362317204.29      1             2024            18
11  ur.016620004052.08      1             1996             1
12     ur.057767065.52      1             2007             4

In [50]:
%%dsldf

search grants where researchers.id in ["ur.0723426172.10", "ur.01277776417.51"] 
return researchers[all]

Returned Researchers: 13


id  count  total_grants  obsolete  first_publication_year  \
0     ur.0723426172.10      8             8         0                  1980.0   
1   ur.012662217132.90      2            13         0                  1972.0   
2   ur.010520247252.54      1             2         0                     NaN   
3    ur.01121653260.31      1            18         0                  1983.0   
4    ur.01127672147.84      1            14         0                  1984.0   
5   ur.011316554452.18      1             1         0                     NaN   
6   ur.012167132327.75      1            11         0                  1987.0   
7   ur.012237141052.77      1             1         0                     NaN   
8   ur.012455520474.57      1             6         0                  1978.0   
9    ur.01274135317.46      1             4         0                  1986.0   
10  ur.013362317204.29      1            18         0                  1977.0   
11  ur.016620004052.08      1             1         0                     NaN   
12     ur.057767065.52      1             4         0                  1961.0   

    last_publication_year      first_name  \
0                  2020.0            Kari   
1                  2020.0  Raymond Philip   
2                     NaN       Raymond P   
3                  2020.0           Teepu   
4                  2020.0          Mark E   
5                     NaN         Richard   
6                  2020.0       Deborah J   
7                     NaN            Mark   
8                  2020.0            Sara   
9                  2020.0       Jeffrey R   
10                 2020.0       Richard J   
11                    NaN       Barry G W   
12                 2019.0       Barry G W   

                                        research_orgs  first_grant_year  \
0   [grid.412578.d, grid.170205.1, grid.415598.4, ...              1984   
1   [grid.266093.8, grid.412578.d, grid.170205.1, ...              1977   
2                                                 NaN              1984   
3   [grid.240684.c, grid.265008.9, grid.83440.3b, ...              1984   
4   [grid.268187.2, grid.214572.7, grid.65456.34, ...              1984   
5                                                 NaN              1984   
6         [grid.5379.8, grid.265892.2, grid.170205.1]              1986   
7                                                 NaN              1988   
8                                     [grid.170205.1]              1979   
9   [grid.421812.c, grid.170205.1, grid.410540.4, ...              2004   
10  [grid.426108.9, grid.170205.1, grid.10698.36, ...              1975   
11                                                NaN              1988   
12  [grid.265008.9, grid.417923.a, grid.412578.d, ...              1977   

   current_research_org                     nih_ppid   last_name  \
0         grid.14013.37  [3321173, 8267810, 3321174]  Stefansson   
1         grid.412578.d  [1865773, 3384368, 3384366]        Roos   
2                   NaN                    [3384367]        Roos   
3         grid.16753.36                    [1889160]    Siddique   
4         grid.438717.e  [2167445, 3346234, 3346233]      Gurney   
5                   NaN                    [3389056]      Miller   
6         grid.170205.1  [1878789, 3229826, 3229825]      Nelson   
7                   NaN                    [3346235]     Gurneyh   
8         grid.170205.1  [3143586, 3412092, 3412091]     Szuchet   
9                   NaN                    [1904928]     Gulcher   
10        grid.16753.36           [3389060, 6232691]      Miller   
11                  NaN                    [3197114]     Arnason   
12                  NaN                    [2418447]     Arnason   

    total_publications  last_grant_year               orcid_id  
0                  639             2018                    NaN  
1                  178             2018  [0000-0002-0613-4048]  
2                    0             2003                

## [DSL-324] Emptiness filter fails on grant.investigator_details

https://uberresearch.atlassian.net/browse/DSL-324

In [51]:
%%dsl
search grants for "malaria" where investigator_details is not empty return grants[investigator_details]

Returned Grants: 20 (total = 10341)


<dimcli.DslDataset object #4806051088. Records: 20/10341>

In [52]:
dsl_last_results.as_dataframe_investigators()

last_name                                       affiliations  \
0        Florin  [{'country_code': 'CH', 'state': None, 'city':...   
1           Hol  [{'city': '', 'name': 'To Be Named North Ameri...   
2          Gill  [{'name': 'SGNT INC', 'country_code': 'US', 'c...   
3     Invernale  [{'name': 'HOUR 72, INC.', 'country_code': 'US...   
4        KAMDEM  [{'city_id': '2220957', 'country': 'CAMEROON',...   
5       Bischof  [{'name': 'University of Minnesota-Twin Cities...   
6       HOFFMAN  [{'city_id': '4367175', 'country': 'UNITED STA...   
7         BAKER  [{'city_id': '4174757', 'country': 'UNITED STA...   
8        DORSEY  [{'city_id': '5391959', 'country': 'UNITED STA...   
9     ROSENTHAL                                                NaN   
10   WEINBERGER  [{'city_id': '5391959', 'country': 'UNITED STA...   
11     MULLIGAN  [{'city_id': '5128581', 'country': 'UNITED STA...   
12          CAO  [{'city_id': '2034937', 'country': 'CHINA', 'c...   
13      CHIBALE  [{'city_id': '7302802', 'country': 'SOUTH AFRI...   
14      FERRINS                                                NaN   
15     ZUERCHER                                                NaN   
16          SUN  [{'city_id': '4843786', 'country': 'UNITED STA...   
17         Lima  [{'state': 'SP', 'state_code': None, 'country_...   
18       Julien                                                NaN   
19         HART  [{'city_id': '5037649', 'country': 'UNITED STA...   
20   GREENHOUSE  [{'city_id': '5391959', 'country': 'UNITED STA...   
21  JAGANNATHAN                                                NaN   
22        BUTTE                                                NaN   
23   Gazzinelli  [{'state': 'SP', 'state_code': None, 'country_...   
24   MAZITSCHEK  [{'city_id': '4931972', 'country': 'UNITED STA...   
25        WIRTH                                                NaN   

                    id       first_name       middle_name   role grant_id  \
0   ur.016225703001.43  Marie-Valentine              None     PI      NaN   
1                 None            Felix                Jh     PI      NaN   
2                 None           Joshua              None     PI      NaN   
3                 None          Michael              None     PI      NaN   
4                 None          COLINCE              None     PI      NaN   
5                 None             John              None     PI      NaN   
6                 None      STEPHEN LEV              None     PI      NaN   
7                 None           BILL J              None     PI      NaN   
8                 None        MATTHEW G              None     PI      NaN   
9                 None       PHILIP JON              None  Co-PI      NaN   
10                None           LEOR S              None     PI      NaN   
11    ur.0633747707.55      MARK JOSEPH              None     PI      NaN   
12   ur.01366702556.41           YAMING              None     PI      NaN   
13   ur.01223360152.24            KELLY              None     PI      NaN   
14    ur.0723077743.12             LORI              None  Co-PI      NaN   
15  ur.014071562725.37          WILLIAM              None  Co-PI      NaN   
16   ur.01361753422.99          JIANJUN              None     PI      NaN   
17                None            Maria  Regina D'Império     PI      NaN   
18                None    Jean-Philippe                       PI      NaN   
19   ur.01025026003.63       GEOFFREY T              None     PI      NaN   
20   ur.01000010550.11          BRYAN R              None     PI      NaN   
21    ur.0766431731.44         PRASANNA              None  Co-PI      NaN   
22    ur.0625713706.08          ATUL J.              None  Co-PI      NaN   
23                None          Ricardo            Tostes     PI      NaN   
24    ur.0654245120.91            RALPH              None     PI      NaN   
25   ur.01251127422.38          DYANN F              None  Co-PI      NaN   

   grant_title grant_start_date gra

## [DSL-269] Inconsistencies in Publications research_org_state_names vs research_org_state_codes

https://uberresearch.atlassian.net/browse/DSL-269

In [53]:
%%dsl
search publications where research_org_state_codes="CA-ON" return publications

Returned Publications: 20 (total = 1174552)


<dimcli.DslDataset object #4806265488. Records: 20/1174552>

In [54]:
%%dsl 
search publications where research_org_state_names="Ontario" 
  return publications

Returned Publications: 20 (total = 1174552)


<dimcli.DslDataset object #4806619408. Records: 20/1174552>

In [55]:
%%dsl 
search publications where research_org_state_names="Ontario" 
  and research_org_state_codes!="CA-ON" 
  return publications[research_org_state_names+research_org_state_codes] limit 100

Returned Publications: 0


<dimcli.DslDataset object #4806619664. Records: 0/0>

## [DSL-457] category_sdg in publications broken

https://uberresearch.atlassian.net/browse/DSL-457

In [56]:
%%dsldf 

search publications return category_sdg

Returned Category_sdg: 17


id    count                                       name
0   40007  1393336              7 Affordable and Clean Energy
1   40003  1312660               3 Good Health and Well Being
2   40016   668961  16 Peace, Justice and Strong Institutions
3   40004   539488                        4 Quality Education
4   40013   442810                          13 Climate Action
5   40008   237138          8 Decent Work and Economic Growth
6   40011   204164      11 Sustainable Cities and Communities
7   40010   170157                    10 Reduced Inequalities
8   40002   124774                              2 Zero Hunger
9   40006    73088               6 Clean Water and Sanitation
10  40014    63860                        14 Life Below Water
11  40015    63233                            15 Life on Land
12  40012    60710  12 Responsible Consumption and Production
13  40001    45273                               1 No Poverty
14  40005    22524                          5 Gender Equality
15  40009    19998  9 Industry, Innovation and Infrastructure
16  40017    10743              17 Partnerships for the Goals

In [57]:
%%dsldf
search publications where category_sdg.name="11 Sustainable Cities and Communities" return publications

Returned Publications: 20 (total = 204164)


year issue     type volume  \
0   2020     1  article     16   
1   2020     1  article     16   
2   2020     1  article      6   
3   2020     1  article      6   
4   2020     1  article      6   
5   2020     1  article      6   
6   2020     1  article      6   
7   2020     2  article     28   
8   2020     6  article    142   
9   2020     2  article      4   
10  2020     1  article     20   
11  2020     1  article     10   
12  2020     1  article     18   
13  2020     1  article      9   
14  2020     1  article      7   
15  2020     1  article     19   
16  2020     1  article     10   
17  2020     1  article      7   
18  2020     1  article     11   
19  2020     1  article     20   

                                  author_affiliations              id  \
0   [[{'first_name': 'Menno', 'last_name': 'Yap', ...  pub.1107667522   
1   [[{'first_name': 'Yunzhe', 'last_name': 'Liu',...  pub.1105205926   
2   [[{'first_name': 'Kangkang', 'last_name': 'Gu'...  pub.1126372661   
3   [[{'first_name': 'Wen', 'last_name': 'Wu', 'ra...  pub.1127576580   
4   [[{'first_name': 'Qicheng', 'last_name': 'Zhon...  pub.1125684642   
5   [[{'first_name': 'Hua', 'last_name': 'Ma', 'ra...  pub.1124300554   
6   [[{'first_name': 'Fangqi', 'last_name': 'Zhao'...  pub.1126182990   
7   [[{'first_name': 'Nikolay K.', 'last_name': 'K...  pub.1128390840   
8   [[{'first_name': 'Sean', 'last_name': 'Lin', '...  pub.1125587329   
9   [[{'first_name': 'Gökçen', 'last_name': 'Firde...  pub.1123969916   
10  [[{'first_name': 'Siri', 'last_name': 'Wiig', ...  pub.1126815036   
11  [[{'first_name': 'Chundi', 'last_name': 'Chen'...  pub.1128558788   
12  [[{'first_name': 'Stephen R.', 'last_name': 'H...  pub.1128669838   
13  [[{'first_name': 'Ghazaleh', 'last_name': 'Rab...  pub.1127883142   
14  [[{'first_name': 'Pilar Maria', 'last_name': '...  pub.1124349387   
15  [[{'first_name': 'Shew Fung', 'last_name': 'Wo...  pub.1126125703   
16  [[{'first_name': 'Naresh D.', 'last_name': 'Sa...  pub.1125616352   
17  [[{'first_name': 'Michele', 'last_name': 'Vale...  pub.1125738864   
18  [[{'first_name': 'Huanbi', 'last_name': 'Yue',...  pub.1125754114   
19  [[{'first_name': 'Rebecca', 'last_name': 'King...  pub.1128556243   

                                                title    pages    journal.id  \
0   Crowding valuation in urban tram and bus trans...     1-20  jour.1145297   
1   Understanding public transit patterns with ope...     1-37  jour.1145297   
2   Spatial planning for urban ventilation corrido...  1747946  jour.1149989   
3   Change of impervious surface area and its impa...  1767511  jour.1149989   
4   A conceptual framework for ex ante valuation o...  1743206  jour.1149989   
5   How does environmental concern influence publi...  1722033  jour.1149989   
6   The compactness of spatial structure in Chines...  1743763  jour.1149989   
7   The accessibility of the center of Moscow as a...  334-346  jour.1299696   
8   Understanding the Effects of Capturing Climate...      NaN  jour.1049907   
9   Evaluation of Aesthetic, Functional, and Envir...    67-86  jour.1298918   
10  Defining the boundaries and operational concep...      330  jour.1327417   
11  Walls offer potential to improve urban biodive...     9905  jour.1045337   
12  How to strengthen a health research system: WH...       72  jour.1032049   
13  Factors affecting severe weather threat index ...        9  jour.1136119   
14  Migration, translation, and transformation of ...        3  jour.1136463   
15  Association between long-term exposure to ambi...       37  jour.1327425   
16  Circular manufacturing of chitinous bio-compos...     4632  jour.1045337   
17  Among territorial project and architectural de...        7  jour.1136463   
18  Stronger policy required to substantially redu...     1462  jour.1043282   
19  A process for developing a sustainable and sca...      950  jour.1024954   

                           journal.title  
0   Transportmetrica A Transpor

## [DSL-378] Funder grant numbers inconsistency

https://uberresearch.atlassian.net/browse/DSL-378

#### Exact match on grants core fails 

In [58]:
%dsldf search grants where grant_number="HL117648" and funders.name="NIH"

Returned Grants: 0
WARNINGS [1]
Please review your query, as it contains an entity filter (funders.name) that can lead to incomplete results. More details on https://docs.dimensions.ai/dsl/language.html#literal-fields-vs-entity-fields


Empty DataFrame
Columns: []
Index: []

#### Partial match, but with funder name, fails 

In [59]:
%dsldf search grants where grant_number~"HL117648" and funders.name="NIH"

Returned Grants: 0
WARNINGS [1]
Please review your query, as it contains an entity filter (funders.name) that can lead to incomplete results. More details on https://docs.dimensions.ai/dsl/language.html#literal-fields-vs-entity-fields


Empty DataFrame
Columns: []
Index: []

#### Partial match, without funder name, succeeds 

In [60]:
%dsl search grants where grant_number~"HL117648"

Returned Grants: 1 (total = 1)


<dimcli.DslDataset object #4806716944. Records: 1/1>

#### Interestingly, 'NIH' doesn't appear anywhere in the funder JSON info

In [61]:
dsl_last_results.grants[0]['funders'][0]

{'id': 'grid.279885.9',
 'types': ['Facility'],
 'name': 'National Heart Lung and Blood Institute',
 'latitude': 39.00428,
 'city_name': 'Bethesda',
 'country_name': 'United States',
 'linkout': ['http://www.nhlbi.nih.gov/'],
 'state_name': 'Maryland',
 'longitude': -77.100945,
 'acronym': 'NHLBI'}

#### Hence also a partial match on the funder name fails 

In [62]:
%dsldf search grants where grant_number~"HL117648" and funders.name~"NIH"

Returned Grants: 0
WARNINGS [1]
Please review your query, as it contains an entity filter (funders.name) that can lead to incomplete results. More details on https://docs.dimensions.ai/dsl/language.html#literal-fields-vs-entity-fields


Empty DataFrame
Columns: []
Index: []

#### Function with NIH as funder name works - maybe this is taken from GRID?

In [63]:
%dsl extract_grants(grant_number="HL117648", funder_name="NIH")


<dimcli.DslDataset object #4806714768. Dict keys: 'grant_id'>

#### Function with wrong funder name fails 

In [64]:
%dsl extract_grants(grant_number="HL117648", funder_name="NH")


<dimcli.DslDataset object #4806778256. Dict keys: 'grant_id'>

#### Looking at GRID, the term NIH is not even in there.. prob it's the parent org

In [65]:
%dsldf search organizations where id="grid.279885.9" return organizations[basics+external_ids_fundref]

Returned Organizations: 1 (total = 1)


latitude       types acronym   country_name external_ids_fundref  \
0  39.00428  [Facility]   NHLBI  United States          [100000050]   

                                      name state_name  longitude  \
0  National Heart Lung and Blood Institute   Maryland -77.100945   

              id                      linkout city_name  
0  grid.279885.9  [http://www.nhlbi.nih.gov/]  Bethesda

#### Function can also work with fundref ID which wouldn't be available via DSL (it's in the org core)

In [66]:
%dsl extract_grants(grant_number="HL117648", fundref="100000050")

<dimcli.DslDataset object #4806928656. Dict keys: 'grant_id'>

## [DSL-414] patent funder countries

https://uberresearch.atlassian.net/browse/DSL-414

In [67]:
%dsldf search patents where funder_countries is not empty return funder_countries

Returned Funder_countries: 1


id   count           name
0  US  181423  United States

In [68]:
%dsldf search patents where funder_countries = "ENG" return patents

Returned Patents: 0


Empty DataFrame
Columns: []
Index: []

In [69]:
%dsldf search patents where funder_countries.name = "United States" return patents

Returned Patents: 20 (total = 181423)


year  times_cited                                          assignees  \
0   2002            1  [{'id': 'grid.453773.1', 'name': 'Wisconsin Al...   
1   2001            0  [{'id': 'grid.16750.35', 'name': 'Princeton Un...   
2   2001            5  [{'id': 'grid.239395.7', 'name': 'Beth Israel ...   
3   2003            0  [{'id': 'grid.30389.31', 'name': 'University o...   
4   2001           27  [{'id': 'grid.265892.2', 'name': 'University o...   
5   2001           10  [{'id': 'grid.4367.6', 'name': 'Washington Uni...   
6   2002            0  [{'id': 'grid.284601.9', 'name': 'IM Systems (...   
7   2001            0  [{'id': 'grid.266900.b', 'name': 'University o...   
8   2001            3  [{'id': 'grid.85084.31', 'name': 'United State...   
9   2001            2  [{'id': 'grid.420747.2', 'name': 'Honeywell (U...   
10  1999           92  [{'id': 'grid.21729.3f', 'name': 'Columbia Uni...   
11  1999            8  [{'id': 'grid.214007.0', 'name': 'Scripps Rese...   
12  2001            0                                                NaN   
13  1996           10  [{'id': 'grid.21729.3f', 'name': 'Columbia Uni...   
14  1999           20  [{'id': 'grid.164295.d', 'name': 'University o...   
15  1999            3  [{'id': 'grid.423597.e', 'name': 'Brookhaven S...   
16  1999            4  [{'id': 'grid.30389.31', 'name': 'University o...   
17  1999            0  [{'id': 'grid.434172.7', 'name': 'Micron Techn...   
18  1999            0  [{'id': 'grid.38142.3c', 'name': 'Harvard Univ...   
19  1996           46                                                NaN   

    granted_year                                     inventor_names  \
0         2003.0  [Brent H. McCown, Eric L. Zeldin, Peter Normin...   
1            NaN                                     [Kevan Shokat]   
2         2003.0                      [Kiflai Bein, Michael Simons]   
3         2003.0                          [Lin Song Li, Quanxi Jia]   
4         2004.0  [Jeffrey A. Engler, Jae Hwy Lee, James F. Coll...   
5         2003.0                [David H. Perlmutter, Nancy Marcus]   
6         2003.0              [David K. Neumann, Thomas L. Henshaw]   
7         2003.0            [Richard D. Cummings, Rodger P. McEver]   
8         2002.0  [Michael A. Hiskey, David E. Chavez, Darren Naud]   
9         2002.0   [Steven E. Gianoulakis, Avijit K. Ray-Chaudhuri]   
10        2002.0              [Jan Kitajewski, Hendrik Uyttendaele]   
11        2002.0             [Chi-Huey Wong, Francisco Moris-Varas]   
12        2003.0                             [Nada H. Saab-Ishmail]   
13        2001.0                    [Paul B. Fisher, Jorge A. Leon]   
14        2001.0  [Suzanne Ostrand-Rosenberg, Sivasubramanian Ba...   
15        2002.0                                 [Paul I. Freimuth]   
16        2001.0  [Mark Tessier-Lavigne, E. David Leonardo, Lind...   
17        2002.0                                   [James J. Alwan]   
18        2001.0              [Stephen M. Beverley, Angela K. Cruz]   
19        2000.0                                 [Jeffery W. Kelly]   

   filing_status                                     assignee_names  \
0          Grant  [Wisconsin Alumni Research Foundation, WISCONS...   
1    Application             [Princeton University, UNIV PRINCETON]   
2          Grant  [Beth Israel Deaconess Medical Center Inc, BET...   
3          Grant        [University of California, UNIV CALIFORNIA]   
4          Grant  [University of Alabama at Birmingham Research ...   
5          Grant  [Washington University in St Louis, UNIV ST LO...   
6          Grant  [Neumann Systems Group Inc, NEUMANN INFORMATIO...   
7          Grant            [University of Oklahoma, UNIV OKLAHOMA]   
8          Grant               [US Department of Energy, US ENERGY]   
9          Grant  [National Technology and Engineering Solutions...   
10         Grant   [Columbia University of New York, UNIV COLUMBIA]   
11         Grant  [Scripps Research Institute, SCRIPPS RESEARCH

## [DSL-311] Upgrade hrcs_rac_v2 

https://uberresearch.atlassian.net/browse/DSL-311

In [70]:
for s in G.sources():
    q = f"""search {s} where category_hrcs_rac is not empty return category_hrcs_rac"""
    dsl.query(q).as_dataframe()

Returned Category_hrcs_rac: 20
Returned Category_hrcs_rac: 20
Returned Category_hrcs_rac: 20
Returned Category_hrcs_rac: 20
Returned Category_hrcs_rac: 20
Returned Errors: 1
Semantic Error
Semantic errors found:
	Facet 'category_hrcs_rac' is not present in source 'researchers'. Available facets are: current_research_org,first_grant_year,first_publication_year,last_grant_year,last_publication_year,research_orgs,total_grants,total_publications
	Field 'category_hrcs_rac' is not present in source 'researchers'. Available fields are: current_research_org,first_grant_year,first_name,first_publication_year,id,last_grant_year,last_name,last_publication_year,nih_ppid,obsolete,orcid_id,redirect,research_orgs,total_grants,total_publications
Returned Errors: 1
Semantic Error
Semantic errors found:
	Facet 'category_hrcs_rac' is not present in source 'organizations'. Available facets are: city_name,country_name,nuts_level1_code,nuts_level1_name,nuts_level2_code,nuts_level2_name,nuts_level3_code,nuts

## [DSL-292] Add ROR IDs 

https://uberresearch.atlassian.net/browse/DSL-292

In [71]:
%dsldf search organizations where ror_ids is not empty return organizations[id+name+ror_ids]

Returned Organizations: 20 (total = 96592)


ror_ids  \
0   [https://ror.org/0121dpf30]   
1   [https://ror.org/03t3qg659]   
2   [https://ror.org/034xvzb47]   
3   [https://ror.org/05rsv9s98]   
4   [https://ror.org/03jmfdf59]   
5   [https://ror.org/042twtr12]   
6   [https://ror.org/0493hgw16]   
7   [https://ror.org/02xey9a22]   
8   [https://ror.org/04byxyr05]   
9   [https://ror.org/040gcmg81]   
10  [https://ror.org/04pw6fb54]   
11  [https://ror.org/00190t495]   
12  [https://ror.org/03wkg3b53]   
13  [https://ror.org/012pb6c26]   
14  [https://ror.org/00baak391]   
15  [https://ror.org/043z4tv69]   
16  [https://ror.org/006zn3t30]   
17  [https://ror.org/00372qc85]   
18  [https://ror.org/004a2wv92]   
19  [https://ror.org/00adh9b73]   

                                                 name             id  
0        Health Resources and Services Administration  grid.454842.b  
1   Substance Abuse and Mental Health Services Adm...  grid.413730.2  
2          United States Food and Drug Administration  grid.417587.8  
3        United States Department of Veterans Affairs  grid.418356.d  
4          Agency for Healthcare Research and Quality  grid.413404.6  
5          Centers for Disease Control and Prevention  grid.416738.f  
6   National Institute on Minority Health and Heal...  grid.281076.a  
7                        Fogarty International Center  grid.453035.4  
8   National Institute of Child Health and Human D...  grid.420089.7  
9                           National Cancer Institute  grid.48336.3a  
10  National Center for Advancing Translational Sc...  grid.429651.d  
11  National Center for Complementary and Integrat...  grid.280655.c  
12                             National Eye Institute  grid.280030.9  
13            National Heart Lung and Blood Institute  grid.279885.9  
14           National Human Genome Research Institute  grid.280128.1  
15  National Institute of Allergy and Infectious D...  grid.419681.3  
16  National Institute of Arthritis and Musculoske...  grid.420086.8  
17  National Institute of Biomedical Imaging and B...  grid.280347.a  
18  National Institute of Dental and Craniofacial ...  grid.419633.a  
19  National Institute of Diabetes and Digestive a...  grid.419635.c

In [72]:
%dsldf search organizations where ror_ids = "https://ror.org/0493hgw16" return organizations[id+name+ror_ids]

Returned Organizations: 1 (total = 1)


ror_ids  \
0  [https://ror.org/0493hgw16]   

                                                name             id  
0  National Institute on Minority Health and Heal...  grid.281076.a

In [73]:
%dsldf search organizations where ror_ids in ["https://ror.org/0493hgw16", "https://ror.org/006zn3t30"] return organizations[id+name+ror_ids]

Returned Organizations: 2 (total = 2)


ror_ids  \
0  [https://ror.org/0493hgw16]   
1  [https://ror.org/006zn3t30]   

                                                name             id  
0  National Institute on Minority Health and Heal...  grid.281076.a  
1  National Institute of Arthritis and Musculoske...  grid.420086.8

In [74]:
%dsldf search organizations return ror_ids

Returned Errors: 1
Semantic Error
Semantic errors found:
	Facet 'ror_ids' is not present in source 'organizations'. Available facets are: city_name,country_name,nuts_level1_code,nuts_level1_name,nuts_level2_code,nuts_level2_name,nuts_level3_code,nuts_level3_name,state_name,types


## [DSL-449] Add is multivalue to describe

https://uberresearch.atlassian.net/browse/DSL-449

In [75]:
z = dsl.query("describe source grants")
for x in z['fields']:
    print(x, z['fields'][x]['is_multivalue'])

funding_cad False
original_title False
funding_org_city False
funding_nzd False
funding_currency False
start_date False
category_hrcs_rac True
concepts True
grant_number False
research_org_cities True
category_bra True
category_sdg True
category_hra True
date_inserted False
investigator_details False
language False
category_rcdc True
category_icrp_cso True
funding_aud False
research_org_state_codes True
language_title False
active_year True
category_hrcs_hc True
research_org_countries True
funding_eur False
category_uoa True
researchers True
abstract False
title False
funding_org_name False
funding_usd False
funders True
start_year False
category_icrp_ct True
category_for True
funding_chf False
research_orgs True
funder_countries True
funding_org_acronym False
funding_gbp False
foa_number False
linkout False
end_date False
funding_jpy False
id False


## [DSL-411] NUTS level 3

https://uberresearch.atlassian.net/browse/DSL-441

In [76]:
%dsldf search organizations return nuts_level3_code 

Returned Nuts_level3_code: 20


id  count
0   UKI32   1964
1   CZ020   1150
2   FR101    956
3   SE110    666
4   DE300    516
5   ES300    479
6   BE100    467
7   CZ064    433
8   ES511    401
9   IE021    391
10  ITI43    384
11  AT130    373
12  DE21H    338
13  PL127    338
14  UKH12    322
15  HU101    300
16  FI1B1    299
17  NL310    288
18  ITC4C    271
19  PT170    263

In [77]:
%dsldf search organizations return nuts_level3_name

Returned Nuts_level3_name: 20


id  count
0                                         Westminster   1964
1                                    Středočeský kraj   1150
2                                               Paris    956
3                                      Stockholms län    666
4                                              Berlin    516
5                                              Madrid    479
6   Arr. de Bruxelles-Capitale / Arr. van Brussel-...    467
7                                   Jihomoravský kraj    433
8                                           Barcelona    401
9                                              Dublin    391
10                                               Roma    384
11                                               Wien    373
12                                    Miasto Warszawa    338
13                                 München, Landkreis    338
14                                  Cambridgeshire CC    322
15                                           Budapest    300
16                                   Helsinki-Uusimaa    299
17                                            Utrecht    288
18                                             Milano    271
19                       Área Metropolitana de Lisboa    263

## [DSL-444] Function expressions: funding_per_year, citations_per_year

https://uberresearch.atlassian.net/browse/DSL-444

https://docs.dimensions.ai/dsl/1.26.0-preview/language.html#returning-complex-calculations-using-result-function-expressions

In [78]:
%dsl search grants for "malaria" return funding_per_year(2010, 2020, "CAD")

Returned Funding_per_year: 11


<dimcli.DslDataset object #4807176272. Dict keys: '_stats', '_version', 'funding_per_year'>

In [79]:
currencies = ['CAD','CHF','EUR','NZD','CNY','GBP','JPY','USD','AUD']

for x in currencies:
    q = f"""search grants for "malaria" return funding_per_year(2010, 2020, "{x}")"""
    print("--", q)
    display(dsl.query(q).funding_per_year)

-- search grants for "malaria" return funding_per_year(2010, 2020, "CAD")
Returned Funding_per_year: 11


{'2010': 714438383.0,
 '2011': 639584017.0,
 '2012': 636685852.0,
 '2013': 680469990.0,
 '2014': 797477635.0,
 '2015': 845116461.0,
 '2016': 907046992.0,
 '2017': 961895216.0,
 '2018': 1002570100.0,
 '2019': 1009336646.0,
 '2020': 763784737.0}

-- search grants for "malaria" return funding_per_year(2010, 2020, "CHF")
Returned Funding_per_year: 11


{'2010': 712034761.0,
 '2011': 601163025.0,
 '2012': 593538936.0,
 '2013': 623430563.0,
 '2014': 681810929.0,
 '2015': 659332212.0,
 '2016': 682002492.0,
 '2017': 729045258.0,
 '2018': 758867462.0,
 '2019': 758368535.0,
 '2020': 571708061.0}

-- search grants for "malaria" return funding_per_year(2010, 2020, "EUR")
Returned Funding_per_year: 11


...

{'2010': 60819072188.0,
 '2011': 52969774460.0,
 '2012': 51488087145.0,
 '2013': 61017229522.0,
 '2014': 75716398290.0,
 '2015': 80064946273.0,
 '2016': 77562338025.0,
 '2017': 82382722361.0,
 '2018': 85879381291.0,
 '2019': 83999138377.0,
 '2020': 63758961958.0}

-- search grants for "malaria" return funding_per_year(2010, 2020, "USD")
Returned Funding_per_year: 11


{'2010': 669425658.0,
 '2011': 632544077.0,
 '2012': 635179423.0,
 '2013': 666380790.0,
 '2014': 739678665.0,
 '2015': 695722790.0,
 '2016': 699163780.0,
 '2017': 742566861.0,
 '2018': 775274558.0,
 '2019': 767161714.0,
 '2020': 585991567.0}

-- search grants for "malaria" return funding_per_year(2010, 2020, "AUD")
Returned Funding_per_year: 11


{'2010': 772338127.0,
 '2011': 648514621.0,
 '2012': 624176921.0,
 '2013': 678057492.0,
 '2014': 801846332.0,
 '2015': 870883646.0,
 '2016': 925662419.0,
 '2017': 969844002.0,
 '2018': 1028127632.0,
 '2019': 1077344636.0,
 '2020': 837545442.0}

In [80]:
topics = ['malaria', 'graphene', ]

for x in topics:
    q = f"""search publications for "{x}" return citations_per_year(2010, 2020)"""
    print("--", q)
    display(dsl.query(q).citations_per_year)

-- search publications for "malaria" return citations_per_year(2010, 2020)
Returned Citations_per_year: 11


{'2010': 403334.0,
 '2011': 466738.0,
 '2012': 535644.0,
 '2013': 605954.0,
 '2014': 677422.0,
 '2015': 740835.0,
 '2016': 809767.0,
 '2017': 906091.0,
 '2018': 971640.0,
 '2019': 1089685.0,
 '2020': 725488.0}

-- search publications for "graphene" return citations_per_year(2010, 2020)
Returned Citations_per_year: 11


{'2010': 123040.0,
 '2011': 209820.0,
 '2012': 324282.0,
 '2013': 456540.0,
 '2014': 641735.0,
 '2015': 906591.0,
 '2016': 1166402.0,
 '2017': 1519509.0,
 '2018': 1729124.0,
 '2019': 2228775.0,
 '2020': 1789468.0}

## [DSL-475] Grants category sdg

https://uberresearch.atlassian.net/browse/DSL-475

In [81]:
%dsldf search grants where category_sdg is not empty return grants

Returned Grants: 20 (total = 557816)


funders  start_year  \
0   [{'id': 'grid.484521.e', 'types': ['Nonprofit'...        2021   
1   [{'id': 'grid.270680.b', 'types': ['Government...        2021   
2   [{'id': 'grid.270680.b', 'types': ['Government...        2021   
3   [{'id': 'grid.270680.b', 'types': ['Government...        2021   
4   [{'id': 'grid.270680.b', 'types': ['Government...        2021   
5   [{'id': 'grid.270680.b', 'types': ['Government...        2021   
6   [{'id': 'grid.270680.b', 'types': ['Government...        2021   
7   [{'id': 'grid.422239.c', 'types': ['Government...        2021   
8   [{'id': 'grid.270680.b', 'types': ['Government...        2021   
9   [{'id': 'grid.270680.b', 'types': ['Government...        2021   
10  [{'id': 'grid.270680.b', 'types': ['Government...        2021   
11  [{'id': 'grid.270680.b', 'types': ['Government...        2021   
12  [{'id': 'grid.270680.b', 'types': ['Government...        2021   
13  [{'id': 'grid.270680.b', 'types': ['Government...        2021   
14  [{'id': 'grid.270680.b', 'types': ['Government...        2021   
15  [{'id': 'grid.270680.b', 'types': ['Government...        2021   
16  [{'id': 'grid.270680.b', 'types': ['Government...        2021   
17  [{'id': 'grid.270680.b', 'types': ['Government...        2021   
18  [{'id': 'grid.270680.b', 'types': ['Government...        2021   
19  [{'id': 'grid.270680.b', 'types': ['Government...        2021   

                                       original_title  \
0   APPROACH to Enriching the Real World Evidence ...   
1   Knowledge Transfer in Global Gender Programmes...   
2   The molecular mechanisms of sex determination ...   
3   Mass cytometry nanotools for intracellular tar...   
4   Understanding the Dynamic Nature of Catalysts ...   
5   Diffiusion of laws addressing anti-LGBT violen...   
6   Origins of Scarcity: Labour and the Metabolism...   
7   Thought Crime in Anglo-American Law and Legal ...   
8   Analyzing emerging patterns of mortality throu...   
9   From the sky to the sea: genomic insights into...   
10  Speaking Freely: Linguistic Domination, Republ...   
11  From Demos to Ethnos: Popular Revolt and Elite...   
12  Evolutionary processes in calcifying organisms...   
13           Advanced Solutions for Asphalt Pavements   
14  Living Together: a study of Hansalim as a mode...   
15   Group effects of news evaluation in social media   
16  Deciphering fundamental constraints on pathoge...   
17  The Metabolic Niche Framework – Linking Metabo...   
18        Parallel Lives: Dependency and Backwardness   
19  Factors driving mother-to-infant microbiota tr...   

                 active_year title_language  start_date language  \
0                     [2021]             en  2021-11-30       en   
1   [2021, 2022, 2023, 2024]             en  2021-10-01       en   
2         [2021, 2022, 2023]             en  2021-09-01       en   
3         [2021, 2022, 2023]             en  2021-09-01       en   
4         [2021, 2022, 2023]             en  2021-09-01       en   
5         [2021, 2022, 2023]             en  2021-09-01       en   
6         [2021, 2022, 2023]             en  2021-09-01       en   
7               [2021, 2022]             en  2021-09-01       en   
8         [2021, 2022, 2023]             en  2021-09-01       en   
9         [2021, 2022, 2023]             en  2021-09-01       en   
10        [2021, 2022, 2023]             en  2021-09-01       en   
11        [2021, 2022, 2023]             en  2021-09-01       en   
12        [2021, 2022, 2023]             en  2021-09-01       en   
13        [2021, 2022, 2023]             en  2021-09-01       en   
14  [2021, 2022, 2023, 2024]             en  2021-09-01       en   
15  [2021, 2022, 2023, 2024]             en  2021-09-01       en   
16        [2021, 2022, 2023]             en  2021-09-01       en   
17        [2021, 2022, 2023]             en  2021-09-01       en   
18  [2021, 2022, 2023, 2024]             en  2021-09-01       en   
19        [2021, 2022, 2023]          

In [82]:
%dsldf search grants return category_sdg

Returned Category_sdg: 17


id   count                                       name
0   40007  148851              7 Affordable and Clean Energy
1   40003  107885               3 Good Health and Well Being
2   40004   88292                        4 Quality Education
3   40013   77467                          13 Climate Action
4   40016   46669  16 Peace, Justice and Strong Institutions
5   40011   33447      11 Sustainable Cities and Communities
6   40008   19349          8 Decent Work and Economic Growth
7   40002   17067                              2 Zero Hunger
8   40010   12567                    10 Reduced Inequalities
9   40012   12246  12 Responsible Consumption and Production
10  40015   11363                            15 Life on Land
11  40014    9659                        14 Life Below Water
12  40006    8344               6 Clean Water and Sanitation
13  40009    3755  9 Industry, Innovation and Infrastructure
14  40001    3631                               1 No Poverty
15  40005    1810                          5 Gender Equality
16  40017    1638              17 Partnerships for the Goals

## [DSL-479] Update datasets license

https://uberresearch.atlassian.net/browse/DSL-479

In [83]:
%dsldf search datasets for "machine learning" return datasets[id+license]

Returned Datasets: 20 (total = 5548)
WARNINGS [1]
Field 'license' is deprecated in favor of license_name. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details


id                                   license.url license.name  \
0    5715196           https://opensource.org/licenses/MIT          MIT   
1   12463832  https://creativecommons.org/licenses/by/4.0/    CC BY 4.0   
2    1604916  https://creativecommons.org/licenses/by/4.0/    CC BY 4.0   
3    1604917  https://creativecommons.org/licenses/by/4.0/    CC BY 4.0   
4   11319911  https://creativecommons.org/licenses/by/4.0/    CC BY 4.0   
5    3793596  https://creativecommons.org/licenses/by/4.0/    CC BY 4.0   
6    3793131  https://creativecommons.org/licenses/by/4.0/    CC BY 4.0   
7    3824103  https://creativecommons.org/licenses/by/4.0/    CC BY 4.0   
8    6215084  https://creativecommons.org/licenses/by/4.0/  CC BY + CC0   
9    6215069  https://creativecommons.org/licenses/by/4.0/  CC BY + CC0   
10   9900956  https://creativecommons.org/licenses/by/4.0/    CC BY 4.0   
11   8960102  https://creativecommons.org/licenses/by/4.0/    CC BY 4.0   
12  11563686  https://creativecommons.org/licenses/by/4.0/    CC BY 4.0   
13   3407182  https://creativecommons.org/licenses/by/4.0/    CC BY 4.0   
14   1606311  https://creativecommons.org/licenses/by/4.0/    CC BY 4.0   
15   7790087  https://creativecommons.org/licenses/by/4.0/    CC BY 4.0   
16   7594967  https://creativecommons.org/licenses/by/4.0/    CC BY 4.0   
17  12053637  https://creativecommons.org/licenses/by/4.0/    CC BY 4.0   
18  12019617  https://creativecommons.org/licenses/by/4.0/    CC BY 4.0   
19   9119699  https://creativecommons.org/licenses/by/4.0/    CC BY 4.0   

    license.value  
0               3  
1               1  
2               1  
3               1  
4               1  
5               1  
6               1  
7               1  
8              47  
9              47  
10              1  
11              1  
12              1  
13              1  
14              1  
15              1  
16              1  
17              1  
18              1  
19              1

In [84]:
%dsldf search datasets for "machine learning" return datasets[id+license_name+license_url]

Returned Datasets: 20 (total = 5548)


license_name                                   license_url        id
0           MIT           https://opensource.org/licenses/MIT   5715196
1     CC BY 4.0  https://creativecommons.org/licenses/by/4.0/  12463832
2     CC BY 4.0  https://creativecommons.org/licenses/by/4.0/   1604916
3     CC BY 4.0  https://creativecommons.org/licenses/by/4.0/   1604917
4     CC BY 4.0  https://creativecommons.org/licenses/by/4.0/  11319911
5     CC BY 4.0  https://creativecommons.org/licenses/by/4.0/   3793596
6     CC BY 4.0  https://creativecommons.org/licenses/by/4.0/   3793131
7     CC BY 4.0  https://creativecommons.org/licenses/by/4.0/   3824103
8   CC BY + CC0  https://creativecommons.org/licenses/by/4.0/   6215084
9   CC BY + CC0  https://creativecommons.org/licenses/by/4.0/   6215069
10    CC BY 4.0  https://creativecommons.org/licenses/by/4.0/   9900956
11    CC BY 4.0  https://creativecommons.org/licenses/by/4.0/   8960102
12    CC BY 4.0  https://creativecommons.org/licenses/by/4.0/  11563686
13    CC BY 4.0  https://creativecommons.org/licenses/by/4.0/   3407182
14    CC BY 4.0  https://creativecommons.org/licenses/by/4.0/   1606311
15    CC BY 4.0  https://creativecommons.org/licenses/by/4.0/   7790087
16    CC BY 4.0  https://creativecommons.org/licenses/by/4.0/   7594967
17    CC BY 4.0  https://creativecommons.org/licenses/by/4.0/  12053637
18    CC BY 4.0  https://creativecommons.org/licenses/by/4.0/  12019617
19    CC BY 4.0  https://creativecommons.org/licenses/by/4.0/   9119699

In [85]:
%dsldf search datasets for "machine learning" return license_name

Returned License_name: 16


id  count
0         CC BY 4.0   3888
1       CC BY + CC0    621
2      CC BY-NC 4.0    538
3               CC0    364
4               MIT     72
5      CC BY-SA 4.0     19
6          GPL 3.0+     13
7        Apache 2.0     10
8      In Copyright      8
9      BSD 3-Clause      3
10         GPL 2.0+      3
11            CC BY      2
12        CC BY 1.0      2
13  CC BY-NC-ND 4.0      2
14              GPL      2
15  CC BY-NC-SA 4.0      1

## [DSL-478] Datasets category sdg

https://uberresearch.atlassian.net/browse/DSL-478

In [86]:
%dsldf search datasets where category_sdg is not empty return datasets

Returned Datasets: 20 (total = 75182)


year                                            authors      id  \
0   2012  [{'name': 'Alfred K. Njamnshi', 'orcid': ''}, ...  354171   
1   2012  [{'name': 'Elodie Descloux', 'orcid': ''}, {'n...  354132   
2   2012  [{'name': 'Kirsten E. Lyke', 'orcid': ''}, {'n...  353865   
3   2012  [{'name': 'Kirsten E. Lyke', 'orcid': ''}, {'n...  353833   
4   2012  [{'name': 'Dieudonné Nkoghe', 'orcid': ''}, {'...  353502   
5   2012  [{'name': 'Laura Helena Andrade', 'orcid': ''}...  353374   
6   2012  [{'name': 'Allen H. Hurlbert', 'orcid': ''}, {...  128386   
7   2012  [{'name': 'Tim M. Daw', 'orcid': ''}, {'name':...  128886   
8   2012  [{'name': 'Xin-Ju Xiao', 'orcid': ''}, {'name'...  129831   
9   2012  [{'name': 'Anna V. Protasio', 'orcid': ''}, {'...  129798   
10  2012  [{'name': 'Rori V. Rohlfs', 'orcid': ''}, {'na...  129111   
11  2012  [{'name': 'Darren M. Soanes', 'orcid': ''}, {'...  129032   
12  2012  [{'name': 'Susanne H. Sheehy', 'orcid': ''}, {...  128506   
13  2012  [{'name': 'Felix C. Mark', 'orcid': ''}, {'nam...  128467   
14  2012  [{'name': 'James W. Rudge', 'orcid': ''}, {'na...  128466   
15  2012  [{'name': 'Ambak K. Rai', 'orcid': ''}, {'name...  129088   
16  2012  [{'name': 'James G. Kahn', 'orcid': ''}, {'nam...  129048   
17  2012  [{'name': 'Justin Komguep Nono', 'orcid': ''},...  128440   
18  2012  [{'name': 'Anna de Breij', 'orcid': ''}, {'nam...  128969   
19  2012  [{'name': 'Ing-Kit Lee', 'orcid': ''}, {'name'...  128405   

                                                title  \
0   Data on subjects investigated in the present s...   
1   Correlations between meteorological variables ...   
2   Intracellular Cytokine Expression to Antigen S...   
3                     CD25hi Depletion Assay Results.   
4   Distribution of suspected and confirmed cases ...   
5   Associations of exposure to crime-related§ tra...   
6   Spatiotemporal Variation in Avian Migration Ph...   
7   To Fish or Not to Fish: Factors at Multiple Sc...   
8   Predicted Disappearance of Cephalantheropsis o...   
9   A Systematically Improved High Quality Genome ...   
10  Familial Identification: Population Structure ...   
11  Genome-wide Transcriptional Profiling of Appre...   
12  Phase Ia Clinical Evaluation of the Safety and...   
13  Mitochondrial Function in Antarctic Notothenii...   
14  Health System Resource Gaps and Associated Mor...   
15  Regulatory T Cells Suppress T Cell Activation ...   
16  Integrated HIV Testing, Malaria, and Diarrhea ...   
17  Excretory/Secretory-Products of Echinococcus m...   
18  Table S1 - Differences in Acinetobacter bauman...   
19  Table S1 - Fatal Dengue Hemorrhagic Fever in A...   

                                             keywords    journal.id  \
0                            [subjects, investigated]  jour.1037994   
1      [meteorological, variables, dengue, outbreaks]  jour.1037994   
2                                 [cytokine, antigen]  jour.1037553   
3                                  [depletion, assay]  jour.1037553   
4   [suspected, confirmed, cases, 2010, chikunguny...  jour.1037994   
5            [traumatic, events, 12-month, disorders]  jour.1037553   
6           [spatiotemporal, avian, reveals, effects]  jour.1037553   
7   [livelihood, conservation policy imperatives, ...  jour.1037553   
8   [predicted, disappearance, luofu, changes, rai...  jour.1037553   
9      [systematically, genome, transcriptome, fluke]  jour.1037994   
10                     [familial, distinguishability]  jour.1034974   
11  [genome-wide, transcriptional, profiling, appr...  jour.1034963   
12  [ia, immunogenicity, blood-stage, antigen, ama...  jour.1037553   
13  [mitochondrial, antarctic, nototheniids, trans...  jour.1037553   
14    [gaps, pandemic, influenza, asian, territories]  jour.1037553   
15  [cells, suppress, activation, pathologic, visc...  jour.1037553   
16       [hiv, diarrhea, modeled, cost-effectiveness]  jour.1037553   
17  [larvae, induce, apoptosis, tole

In [87]:
%dsldf search datasets return category_sdg

Returned Category_sdg: 17


id  count                                       name
0   40003  29150               3 Good Health and Well Being
1   40013  18434                          13 Climate Action
2   40007   9283              7 Affordable and Clean Energy
3   40008   3369          8 Decent Work and Economic Growth
4   40014   3176                        14 Life Below Water
5   40004   2979                        4 Quality Education
6   40016   2308  16 Peace, Justice and Strong Institutions
7   40006   2016               6 Clean Water and Sanitation
8   40002   1850                              2 Zero Hunger
9   40015   1303                            15 Life on Land
10  40011   1010      11 Sustainable Cities and Communities
11  40010    920                    10 Reduced Inequalities
12  40001    398                               1 No Poverty
13  40012    373  12 Responsible Consumption and Production
14  40005    279                          5 Gender Equality
15  40009    102  9 Industry, Innovation and Infrastructure
16  40017     34              17 Partnerships for the Goals

## [DSL-477] Clinical trails mesh terms

https://uberresearch.atlassian.net/browse/DSL-477

In [88]:
%dsldf search clinical_trials where mesh_terms is not empty return clinical_trials[id+mesh_terms]

Returned Clinical_trials: 20 (total = 284983)


mesh_terms           id
0                           [Head and Neck Neoplasms]  NCT02318277
1           [Substance Withdrawal Syndrome, Syndrome]  NCT02318290
2   [Rhinitis, Rhinitis, Allergic, Rhinitis, Aller...  NCT02318303
3   [Urinary Bladder Neoplasms, Carcinoma, Transit...  NCT02318329
4                                        [Thrombosis]  NCT02318342
5                   [Wounds and Injuries, Hemorrhage]  NCT02318355
6    [Carcinoma, Non-Small-Cell Lung, Lung Neoplasms]  NCT02318368
7   [Hereditary Sensory and Motor Neuropathy, Nerv...  NCT02318381
8                                         [Neoplasms]  NCT02318394
9              [Osteoarthritis, Osteoarthritis, Knee]  NCT02318407
10  [Stillbirth, Maternal Death, Obstetric Labor C...  NCT02318420
11                 [Osteoarthritis, Radius Fractures]  NCT02318433
12                                         [Epilepsy]  NCT02318446
13      [Muscle Weakness, Gait Disorders, Neurologic]  NCT02318459
14           [Venous Thrombosis, Thrombosis, Rupture]  NCT02318472
15                [Myocardial Infarction, Infarction]  NCT02318498
16                                   [Osteoarthritis]  NCT02318511
17                   [Lung Diseases, Fungal, Mycoses]  NCT02318524
18                [Syndrome, Lennox Gastaut Syndrome]  NCT02318537
19  [Syndrome, Spasms, Infantile, Epilepsies, Myoc...  NCT02318563

In [89]:
%dsldf search clinical_trials return mesh_terms

Returned Mesh_terms: 20


id  count
0                Diabetes Mellitus   7871
1                 Breast Neoplasms   7790
2                         Syndrome   7182
3        Diabetes Mellitus, Type 2   5967
4                        Neoplasms   5624
5                        Carcinoma   5488
6                   Lung Neoplasms   5021
7                        Infection   4956
8                         Leukemia   4939
9                       Depression   4808
10                    Hypertension   4775
11                        Lymphoma   4725
12             Prostatic Neoplasms   4227
13         Coronary Artery Disease   3903
14  Carcinoma, Non-Small-Cell Lung   3865
15                         Disease   3851
16             Depressive Disorder   3839
17                  HIV Infections   3699
18                          Stroke   3588
19                   Heart Failure   3554

In [90]:
%dsldf search clinical_trials where mesh_terms = "Stroke" return category_for

Returned Category_for: 20


id  count                                               name
0   2211   3385                     11 Medical and Health Sciences
1   3053   1579                             1103 Clinical Sciences
2   3120   1444                                 1109 Neurosciences
3   3177    816             1117 Public Health and Health Services
4   3048    519    1102 Cardiorespiratory Medicine and Haematology
5   2217    311               17 Psychology and Cognitive Sciences
6   3468    309                                    1701 Psychology
7   3097     65             1106 Human Movement and Sports Science
8   3164     57      1115 Pharmacology and Pharmaceutical Sciences
9   2208     53              08 Information and Computing Sciences
10  2746     51  0801 Artificial Intelligence and Image Processing
11  2209     22                                     09 Engineering
12  2837     20                        0903 Biomedical Engineering
13  2206     13                             06 Biological Sciences
14  3079     12        1104 Complementary and Alternative Medicine
15  2620     11                                      0604 Genetics
16  3128      8                                       1110 Nursing
17  3103      7                                    1107 Immunology
18  3142      4                   1112 Oncology and Carcinogenesis
19  3158      4         1114 Paediatrics and Reproductive Medicine

## [DSL-284] Drill down syntax

https://uberresearch.atlassian.net/browse/DSL-284

### 1) If you don't return any publications data, publications without researchers lead to lots of empty rows. We decided to leave as is for now

In [91]:
%%dsldf
search publications return publications[unnest(researchers)]

Returned Publications: 32 (total = 110778116)


researchers.id   researchers.first_name researchers.last_name  \
0                  NaN                      NaN                   NaN   
1                  NaN                      NaN                   NaN   
2                  NaN                      NaN                   NaN   
3    ur.07430064243.75                  Nathaly          Aya Pastrana   
4     ur.0722036447.03                   Claire            Somerville   
5     ur.0663400406.47                L Suzanne                 Suggs   
6   ur.015114237435.63               Mahendra M                 Reddy   
7     ur.0670077515.38                   Pruthu               Thekkur   
8   ur.015065241447.02                   Nagesh                 Ramya   
9   ur.010761343641.82             Prasanna B T                Kamath   
10    ur.0746304355.66                 Suresh G               Shastri   
11   ur.01124645367.27                Palanivel            Chinnakali   
12  ur.013042103661.54         Abhay Subhashrao               Nirgude   
13    ur.0664030760.88             Narasimhaiah            Somashekar   
14  ur.013130304214.27          Benjamin-Samuel              Schlüter   
15   ur.01065454262.07                    Bruno            Masquelier   
16  ur.016452515157.97  Charlotte Jessica Eland               Metcalf   
17                 NaN                      NaN                   NaN   
18    ur.0644571555.93                    Georg             Satzinger   
19   ur.01137217365.42             Hans-Joachim              Ziegeler   
20                 NaN                      NaN                   NaN   
21                 NaN                      NaN                   NaN   
22                 NaN                      NaN                   NaN   
23                 NaN                      NaN                   NaN   
24                 NaN                      NaN                   NaN   
25                 NaN                      NaN                   NaN   
26                 NaN                      NaN                   NaN   
27                 NaN                      NaN                   NaN   
28                 NaN                      NaN                   NaN   
29                 NaN                      NaN                   NaN   
30                 NaN                      NaN                   NaN   
31                 NaN                      NaN                   NaN   

     researchers.orcid_id                          researchers.research_orgs  
0                     NaN                                                NaN  
1                     NaN                                                NaN  
2                     NaN                                                NaN  
3   [0000-0002-0321-2397]                                    [grid.29078.34]  
4                     NaN                                    [grid.424404.2]  
5                     NaN  [grid.264797.9, grid.483659.5, grid.7445.2, gr...  
6   [0000-0001-9726-6064]                     [grid.414953.e, grid.416263.4]  
7   [0000-0001-5257-5074]  [grid.8195.5, grid.414953.e, grid.483403.8, gr...  
8   [0000-0001-6319-7275]                                                NaN  
9                     NaN                                                NaN  
10                    NaN  [grid.464881.7, grid.454780.a, grid.452679.b, ...  
11  [0000-0001-8320-1016]      [grid.414611.7, grid.414953.e, grid.198530.6]  
12                    NaN      [grid.419776.a, grid.416738.f, grid.413027.3]  
13                    NaN                                    [grid.419776.a]  
14                    NaN                                      [grid.7942.8]  
15  [0000-0002-9585-891X]                       [grid.77048.3c, grid.7942.8]  
16                    NaN  [grid.29857.31, grid.16750.35, grid.420089.7, ...  
17                    NaN                                                NaN  
18                    NaN                       [grid.6190.e, grid.10388.32]  
19                

In this case it's ok

In [92]:
%%dsldf
search publications return publications[doi+unnest(researchers)]

Returned Publications: 32 (total = 110778116)


doi      researchers.id  \
0       10.1515/9789048550531-003                 NaN   
1       10.1515/9783110823547-013                 NaN   
2           10.1515/9783110907452                 NaN   
3   10.1080/16549716.2019.1711335   ur.07430064243.75   
4   10.1080/16549716.2019.1711335    ur.0722036447.03   
5   10.1080/16549716.2019.1711335    ur.0663400406.47   
6   10.1080/16549716.2019.1704540  ur.015114237435.63   
7   10.1080/16549716.2019.1704540    ur.0670077515.38   
8   10.1080/16549716.2019.1704540  ur.015065241447.02   
9   10.1080/16549716.2019.1704540  ur.010761343641.82   
10  10.1080/16549716.2019.1704540    ur.0746304355.66   
11  10.1080/16549716.2019.1704540   ur.01124645367.27   
12  10.1080/16549716.2019.1704540  ur.013042103661.54   
13  10.1080/16549716.2019.1704540    ur.0664030760.88   
14  10.1080/16549716.2020.1717411  ur.013130304214.27   
15  10.1080/16549716.2020.1717411   ur.01065454262.07   
16  10.1080/16549716.2020.1717411  ur.016452515157.97   
17      10.1515/9783110950762-012                 NaN   
18      10.1515/9783110922035-011    ur.0644571555.93   
19      10.1515/9783110922035-011   ur.01137217365.42   
20      10.1515/9783110922035-006                 NaN   
21      10.1515/9783110939996-010                 NaN   
22      10.1515/9783110841794-004                 NaN   
23      10.1515/9783110832181-026                 NaN   
24      10.1515/9789048550531-011                 NaN   
25      10.1515/9783110841794-008                 NaN   
26      10.1515/9783110953909-015                 NaN   
27      10.1515/9783110939996-015                 NaN   
28      10.1515/9783110934212-010                 NaN   
29      10.1515/9783110934212-011                 NaN   
30      10.1515/9783110950762-017                 NaN   
31      10.1515/9783110823547-011                 NaN   

     researchers.first_name researchers.last_name   researchers.orcid_id  \
0                       NaN                   NaN                    NaN   
1                       NaN                   NaN                    NaN   
2                       NaN                   NaN                    NaN   
3                   Nathaly          Aya Pastrana  [0000-0002-0321-2397]   
4                    Claire            Somerville                    NaN   
5                 L Suzanne                 Suggs                    NaN   
6                Mahendra M                 Reddy  [0000-0001-9726-6064]   
7                    Pruthu               Thekkur  [0000-0001-5257-5074]   
8                    Nagesh                 Ramya  [0000-0001-6319-7275]   
9              Prasanna B T                Kamath                    NaN   
10                 Suresh G               Shastri                    NaN   
11                Palanivel            Chinnakali  [0000-0001-8320-1016]   
12         Abhay Subhashrao               Nirgude                    NaN   
13             Narasimhaiah            Somashekar                    NaN   
14          Benjamin-Samuel              Schlüter                    NaN   
15                    Bruno            Masquelier  [0000-0002-9585-891X]   
16  Charlotte Jessica Eland               Metcalf                    NaN   
17                      NaN                   NaN                    NaN   
18                    Georg             Satzinger                    NaN   
19             Hans-Joachim              Ziegeler                    NaN   
20                      NaN                   NaN                    NaN   
21                      NaN                   NaN                    NaN   
22                      NaN                   NaN                    NaN   
23                      NaN                   NaN                    NaN   
24                      NaN                   NaN                    NaN   
25                      NaN                   NaN                    NaN   
26                      NaN                   NaN                    NaN   
27                  

### 2) Unnesting category_uoa or category_hra  

In [98]:
%%dsldf
search publications return publications[doi+title+unnest(category_uoa)]

Returned Publications: 20 (total = 110778116)


doi  \
0       10.1515/9789048550531-003   
1       10.1515/9783110823547-013   
2           10.1515/9783110907452   
3   10.1080/16549716.2019.1711335   
4   10.1080/16549716.2019.1704540   
5   10.1080/16549716.2020.1717411   
6       10.1515/9783110950762-012   
7       10.1515/9783110922035-011   
8       10.1515/9783110922035-006   
9       10.1515/9783110939996-010   
10      10.1515/9783110841794-004   
11      10.1515/9783110832181-026   
12      10.1515/9789048550531-011   
13      10.1515/9783110841794-008   
14      10.1515/9783110953909-015   
15      10.1515/9783110939996-015   
16      10.1515/9783110934212-010   
17      10.1515/9783110934212-011   
18      10.1515/9783110950762-017   
19      10.1515/9783110823547-011   

                                                title category_uoa.id  \
0                    2. The Sunflowers in Perspective             NaN   
1                                          Literature             NaN   
2              Die Standesordnungen der freien Berufe             NaN   
3   The gender responsiveness of social marketing ...           30003   
4   To start or to complete? – Challenges in imple...           30002   
5   Long-term trends in seasonality of mortality i...           30002   
6   Eine Warnung an alle, dy sych etwaz duncken: D...             NaN   
7                              Marienklagen und Pietà             NaN   
8                     Johannes Taulers Via negationis             NaN   
9                 Die editorische Einheit ,Textstufe'             NaN   
10                                ad Iliadis librum Ζ             NaN   
11                             De dato patris luminum             NaN   
12  Experimental Methods and Conditions Used for I...             NaN   
13                       In volumine primo corrigenda             NaN   
14                    Index grammaticus et rhetoricus             NaN   
15  Die Darstellung von Textunsicherheiten und nic...             NaN   
16  Zur Geschichte von Georg Hermanns Nachlaß. Ein...             NaN   
17                                             Anhang             NaN   
18  Vom Mittelhochdeutschen zum Frühneuhochdeutsch...             NaN   
19  Representations of Skalds in the Sagas 1: Soci...             NaN   

                                    category_uoa.name  
0                                                 NaN  
1                                                 NaN  
2                                                 NaN  
3   A03 Allied Health Professions, Dentistry, Nurs...  
4   A02 Public Health, Health Services and Primary...  
5   A02 Public Health, Health Services and Primary...  
6                                                 NaN  
7                                                 NaN  
8                                                 NaN  
9                                                 NaN  
10                                                NaN  
11                                                NaN  
12                                                NaN  
13                                                NaN  
14                                                NaN  
15                                                NaN  
16                                                NaN  
17                                                NaN  
18                                                NaN  
19                                                NaN

In [99]:
%%dsldf
search publications return publications[doi+title+unnest(category_hra)]

Returned Publications: 20 (total = 110778116)


doi  \
0       10.1515/9789048550531-003   
1       10.1515/9783110823547-013   
2           10.1515/9783110907452   
3   10.1080/16549716.2019.1711335   
4   10.1080/16549716.2019.1704540   
5   10.1080/16549716.2020.1717411   
6       10.1515/9783110950762-012   
7       10.1515/9783110922035-011   
8       10.1515/9783110922035-006   
9       10.1515/9783110939996-010   
10      10.1515/9783110841794-004   
11      10.1515/9783110832181-026   
12      10.1515/9789048550531-011   
13      10.1515/9783110841794-008   
14      10.1515/9783110953909-015   
15      10.1515/9783110939996-015   
16      10.1515/9783110934212-010   
17      10.1515/9783110934212-011   
18      10.1515/9783110950762-017   
19      10.1515/9783110823547-011   

                                                title category_hra.id  \
0                    2. The Sunflowers in Perspective             NaN   
1                                          Literature             NaN   
2              Die Standesordnungen der freien Berufe             NaN   
3   The gender responsiveness of social marketing ...            3903   
4   To start or to complete? – Challenges in imple...            3903   
5   Long-term trends in seasonality of mortality i...            3903   
6   Eine Warnung an alle, dy sych etwaz duncken: D...             NaN   
7                              Marienklagen und Pietà             NaN   
8                     Johannes Taulers Via negationis             NaN   
9                 Die editorische Einheit ,Textstufe'             NaN   
10                                ad Iliadis librum Ζ             NaN   
11                             De dato patris luminum             NaN   
12  Experimental Methods and Conditions Used for I...             NaN   
13                       In volumine primo corrigenda             NaN   
14                    Index grammaticus et rhetoricus             NaN   
15  Die Darstellung von Textunsicherheiten und nic...             NaN   
16  Zur Geschichte von Georg Hermanns Nachlaß. Ein...             NaN   
17                                             Anhang             NaN   
18  Vom Mittelhochdeutschen zum Frühneuhochdeutsch...             NaN   
19  Representations of Skalds in the Sagas 1: Soci...             NaN   

       category_hra.name  
0                    NaN  
1                    NaN  
2                    NaN  
3   Population & Society  
4   Population & Society  
5   Population & Society  
6                    NaN  
7                    NaN  
8                    NaN  
9                    NaN  
10                   NaN  
11                   NaN  
12                   NaN  
13                   NaN  
14                   NaN  
15                   NaN  
16                   NaN  
17                   NaN  
18                   NaN  
19                   NaN

In [103]:
dsl_last_results.groupby('category_hra.name').count()

doi  title  category_hra.id
category_hra.name                                
Population & Society    3      3                3

### 3) Unnesting funders  

In [104]:
%%dsldf
search publications for "solar cells" return publications[doi+title+unnest(funders)]

Returned Publications: 20 (total = 1280149)


doi  \
0              10.7591/9781501734977-013   
1          10.1080/16549716.2020.1760490   
2             10.36019/9780813595948-007   
3             10.11129/9783955534578-014   
4              10.1515/9783050088570-102   
5              10.1515/9780691202808-009   
6              10.1515/9780691202808-010   
7              10.1515/9780691202808-011   
8              10.1515/9783050078038-006   
9   10.15828/2075-8545-2019-11-6-719-729   
10                  10.4491/eer.2019.387   
11                  10.4491/eer.2019.432   
12    10.22363/2313-0660-2020-20-1-67-83   
13          10.35940/ijitee.b6601.129219   
14                10.1680/jnaen.19.00036   
15                10.1680/jnaen.19.00059   
16            10.1016/j.njas.2020.100327   
17             10.1186/s41938-019-0201-1   
18            10.1186/s11671-020-03330-z   
19            10.1186/s42269-020-00359-z   

                                                title     funders.id  \
0          11. Technologies of Power: The Andean Case            NaN   
1   Eco-epidemiological assessment of the COVID-19...            NaN   
2   5. Genetic Research on Chronic, Severe Mental ...            NaN   
3                         Powerhouse Kjørbo Bærum, NO            NaN   
4                 Von Engels redigierte Übersetzungen            NaN   
5                                          I. JAINISM            NaN   
6                                II. SĀṄKHYA AND YOGA            NaN   
7                                     III. BRĀHMANISM            NaN   
8   Heft 2. Exzerpte zur Geschichte Spaniens aus W...            NaN   
9   The inventions in nanotechnologies as practica...            NaN   
10  Comments on waste to energy technologies in th...  grid.419696.5   
11  Modelling inactivation kinetics of waterborne ...            NaN   
12  South Africa’s Shrinking Sovereignty: Economic...            NaN   
13  Modeling and Simulation of Hybrid Micro Grid E...            NaN   
14  Investigation of Heat Transfer Performance and...            NaN   
15  Natural Dye Sensitized Photo Electrochemical C...            NaN   
16  Estimating crop yield supply responses to be u...            NaN   
17  Increasing the efficacy of the cotton leaf wor...    grid.5018.c   
18  Strain Engineering on the Electronic and Optic...            NaN   
19  Effect of garlic extract, ascorbic acid and ni...            NaN   

                                    funders.name funders.acronym  \
0                                            NaN             NaN   
1                                            NaN             NaN   
2                                            NaN             NaN   
3                                            NaN             NaN   
4                                            NaN             NaN   
5                                            NaN             NaN   
6                                            NaN             NaN   
7                                            NaN             NaN   
8                                            NaN             NaN   
9                                            NaN             NaN   
10  National Natural Science Foundation of China            NSFC   
11                                           NaN             NaN   
12                                           NaN             NaN   
13                                           NaN             NaN   
14                                           NaN             NaN   
15                                           NaN             NaN   
16                                           NaN             NaN   
17                 Hungarian Academy of Sciences             MTA   
18                                           NaN             NaN   
19                                           NaN             NaN   

                              funders.linkout funders.country_name  \
0                                         NaN                  NaN   
1                             

### 4) Unnesting research_org_countries 

In [105]:
%%dsldf
search publications for "solar cells" return publications[doi+title+unnest(research_org_countries)]

Returned Publications: 22 (total = 1280149)


doi  \
0              10.7591/9781501734977-013   
1          10.1080/16549716.2020.1760490   
2          10.1080/16549716.2020.1760490   
3          10.1080/16549716.2020.1760490   
4             10.36019/9780813595948-007   
5             10.11129/9783955534578-014   
6              10.1515/9783050088570-102   
7              10.1515/9780691202808-009   
8              10.1515/9780691202808-010   
9              10.1515/9780691202808-011   
10             10.1515/9783050078038-006   
11  10.15828/2075-8545-2019-11-6-719-729   
12                  10.4491/eer.2019.387   
13                  10.4491/eer.2019.432   
14    10.22363/2313-0660-2020-20-1-67-83   
15          10.35940/ijitee.b6601.129219   
16                10.1680/jnaen.19.00036   
17                10.1680/jnaen.19.00059   
18            10.1016/j.njas.2020.100327   
19             10.1186/s41938-019-0201-1   
20            10.1186/s11671-020-03330-z   
21            10.1186/s42269-020-00359-z   

                                                title  \
0          11. Technologies of Power: The Andean Case   
1   Eco-epidemiological assessment of the COVID-19...   
2   Eco-epidemiological assessment of the COVID-19...   
3   Eco-epidemiological assessment of the COVID-19...   
4   5. Genetic Research on Chronic, Severe Mental ...   
5                         Powerhouse Kjørbo Bærum, NO   
6                 Von Engels redigierte Übersetzungen   
7                                          I. JAINISM   
8                                II. SĀṄKHYA AND YOGA   
9                                     III. BRĀHMANISM   
10  Heft 2. Exzerpte zur Geschichte Spaniens aus W...   
11  The inventions in nanotechnologies as practica...   
12  Comments on waste to energy technologies in th...   
13  Modelling inactivation kinetics of waterborne ...   
14  South Africa’s Shrinking Sovereignty: Economic...   
15  Modeling and Simulation of Hybrid Micro Grid E...   
16  Investigation of Heat Transfer Performance and...   
17  Natural Dye Sensitized Photo Electrochemical C...   
18  Estimating crop yield supply responses to be u...   
19  Increasing the efficacy of the cotton leaf wor...   
20  Strain Engineering on the Electronic and Optic...   
21  Effect of garlic extract, ascorbic acid and ni...   

   research_org_countries.id research_org_countries.name  
0                        NaN                         NaN  
1                         GB              United Kingdom  
2                         SE                      Sweden  
3                         ZA                South Africa  
4                        NaN                         NaN  
5                        NaN                         NaN  
6                        NaN                         NaN  
7                        NaN                         NaN  
8                        NaN                         NaN  
9                        NaN                         NaN  
10                       NaN                         NaN  
11                       NaN                         NaN  
12                       NaN                         NaN  
13                       NaN                         NaN  
14                        ZA                South Africa  
15                       NaN                         NaN  
16                        TN                     Tunisia  
17                        IN                       India  
18                        NL                 Netherlands  
19                        EG                       Egypt  
20                        CN                       China  
21                        EG                       Egypt

### 5) Query generator for all entity+multivalue fields - unnest single field + ID

In [118]:
sources = G.sources()

def get_fields(source):
    "get all entity+multivalue fields for a source"
    res = []
    z = dsl.query(f"describe source {source}")
    for x in z['fields']:
        if z['fields'][x]['is_multivalue'] and z['fields'][x]['is_entity']:
            res += [x]
    return res

q = """search {} where {} is not empty return {}[id+unnest({})]"""

for s in sources:
    print("\n***********************")
    print("SOURCE:", s)
    print("***********************")
    fields = get_fields(s)
    print("Fields: ", fields)
    for f in fields:
        query = q.format(s, f, s, f)
        print("\n====\n", query, "\n====\n")
        try:
            display(dsl.query(query).as_dataframe().head(5))
        except:
            print("Error")
        time.sleep(1)
        



***********************
SOURCE: publications
***********************
Fields:  ['category_hrcs_rac', 'open_access_categories', 'research_org_cities', 'category_bra', 'category_sdg', 'category_hra', 'category_rcdc', 'category_icrp_cso', 'research_org_state_codes', 'category_hrcs_hc', 'research_org_countries', 'category_uoa', 'researchers', 'funders', 'category_icrp_ct', 'category_for', 'funder_countries', 'research_orgs']

====
 search publications where category_hrcs_rac is not empty return publications[id+unnest(category_hrcs_rac)] 
====

Returned Publications: 44 (total = 7428858)


id category_hrcs_rac.id  \
0  pub.1124196727                10204   
1  pub.1124196727                10200   
2  pub.1124099280                10800   
3  pub.1124099280                10801   
4  pub.1124012124                10606   

                       category_hrcs_rac.name  
0           2.4 Surveillance and distribution  
1                                 2 Aetiology  
2  8 Health and Social Care Services Research  
3   8.1 Organisation and delivery of services  
4           6.6 Psychological and behavioural


====
 search publications where open_access_categories is not empty return publications[id+unnest(open_access_categories)] 
====

Returned Publications: 23 (total = 110778115)


id open_access_categories.id open_access_categories.name  \
0  pub.1125801740                    closed                      Closed   
1  pub.1125632078                    closed                      Closed   
2  pub.1096916023                    closed                      Closed   
3  pub.1124196727                    oa_all                      All OA   
4  pub.1124196727               gold_bronze                      Bronze   

                  open_access_categories.description  
0       No freely available copy has been identified  
1       No freely available copy has been identified  
2       No freely available copy has been identified  
3                        Article is freely available  
4  Freely available on publisher page, but withou...


====
 search publications where research_org_cities is not empty return publications[id+unnest(research_org_cities)] 
====

Returned Publications: 64 (total = 56237927)


...

id category_for.id                              category_for.name
0  354605            2211                 11 Medical and Health Sciences
1  354605            3164  1115 Pharmacology and Pharmaceutical Sciences
2  354569            2211                 11 Medical and Health Sciences
3  354569            3053                         1103 Clinical Sciences
4  354340            2211                 11 Medical and Health Sciences


====
 search datasets where funder_countries is not empty return datasets[id+unnest(funder_countries)] 
====

Returned Datasets: 27 (total = 673883)


id funder_countries.id funder_countries.name
0  354171                  US         United States
1  354171                  GB        United Kingdom
2  353996                  CA                Canada
3  353964                  CA                Canada
4  353934                  CA                Canada


====
 search datasets where research_orgs is not empty return datasets[id+unnest(research_orgs)] 
====

Returned Datasets: 56 (total = 1069454)


id research_orgs.id                              research_orgs.name  \
0  354641    grid.11136.34                         University of Perpignan   
1  354641      grid.4444.0  French National Centre for Scientific Research   
2  354641    grid.462228.8           Laboratoire de Chimie de Coordination   
3  354641    grid.503395.9                                Palumed (France)   
4  354605    grid.11136.34                         University of Perpignan   

                         research_orgs.linkout research_orgs.country_name  \
0                   [http://www.univ-perp.fr/]                     France   
1               [http://www.cnrs.fr/index.php]                     France   
2    [http://www.lcc-toulouse.fr/lcc/?lang=en]                     France   
3  [http://lasersprint.com/palumed/?article15]                     France   
4                   [http://www.univ-perp.fr/]                     France   

  research_orgs.state_name research_orgs.city_name  research_orgs.latitude  \
0     Languedoc-Roussillon               Perpignan               42.683300   
1                      NaN                   Paris               48.847664   
2                      NaN                Toulouse               43.557655   
3                      NaN        Castanet-Tolosan               43.512910   
4     Languedoc-Roussillon               Perpignan               42.683300   

   research_orgs.longitude research_orgs.types research_orgs.acronym  
0                 2.901670         [Education]                   NaN  
1                 2.264034        [Government]                  CNRS  
2                 1.463439          [Facility]                   LCC  
3                 1.521538           [Company]                   NaN  
4                 2.901670         [Education]                   NaN

### 6) Query generator for all entity+multivalue fields - unnest various fields at the same time

Each field is added every iteration to the return unnest statement..

In [126]:
sources = G.sources()

def get_fields(source):
    "get all entity+multivalue fields for a source"
    res = []
    z = dsl.query(f"describe source {source}")
    for x in z['fields']:
        if z['fields'][x]['is_multivalue'] and z['fields'][x]['is_entity']:
            res += [x]
    return res

q = """search {} where {} is not empty return {}[id{}]"""

for s in sources:
    print("\n***********************")
    print("SOURCE:", s)
    print("***********************")
    fields = get_fields(s)
    print("Fields: ", fields)
    unnest_bit = ""
    for f in fields:
        unnest_bit += f"+unnest({f})"
        query = q.format(s, f, s, unnest_bit)
        print("\n====\n", query, "\n====\n")
        try:
            display(dsl.query(query).as_dataframe().head(5))
        except:
            print("Error")
        time.sleep(1)
        



***********************
SOURCE: publications
***********************
Fields:  ['category_hrcs_rac', 'open_access_categories', 'research_org_cities', 'category_bra', 'category_sdg', 'category_hra', 'category_rcdc', 'category_icrp_cso', 'research_org_state_codes', 'category_hrcs_hc', 'research_org_countries', 'category_uoa', 'researchers', 'funders', 'category_icrp_ct', 'category_for', 'funder_countries', 'research_orgs']

====
 search publications where category_hrcs_rac is not empty return publications[id+unnest(category_hrcs_rac)] 
====

Returned Publications: 44 (total = 7428858)


id category_hrcs_rac.id  \
0  pub.1124196727                10204   
1  pub.1124196727                10200   
2  pub.1124099280                10800   
3  pub.1124099280                10801   
4  pub.1124012124                10606   

                       category_hrcs_rac.name  
0           2.4 Surveillance and distribution  
1                                 2 Aetiology  
2  8 Health and Social Care Services Research  
3   8.1 Organisation and delivery of services  
4           6.6 Psychological and behavioural


====
 search publications where open_access_categories is not empty return publications[id+unnest(category_hrcs_rac)+unnest(open_access_categories)] 
====

Returned Publications: 27 (total = 110778115)


id open_access_categories.id open_access_categories.name  \
0  pub.1125801740                    closed                      Closed   
1  pub.1125632078                    closed                      Closed   
2  pub.1096916023                    closed                      Closed   
3  pub.1124196727                    oa_all                      All OA   
4  pub.1124196727                    oa_all                      All OA   

             open_access_categories.description category_hrcs_rac.id  \
0  No freely available copy has been identified                  NaN   
1  No freely available copy has been identified                  NaN   
2  No freely available copy has been identified                  NaN   
3                   Article is freely available                10204   
4                   Article is freely available                10200   

              category_hrcs_rac.name  
0                                NaN  
1                                NaN  
2                                NaN  
3  2.4 Surveillance and distribution  
4                        2 Aetiology


====
 search publications where research_org_cities is not empty return publications[id+unnest(category_hrcs_rac)+unnest(open_access_categories)+unnest(research_org_cities)] 
====

Returned Publications: 193 (total = 56237927)


...

id     researchers.id researchers.first_name researchers.last_name  \
0  354605  ur.01011577376.25                 Julien               Portela   
1  354605  ur.01011577376.25                 Julien               Portela   
2  354605  ur.01011577376.25                 Julien               Portela   
3  354605  ur.01011577376.25                 Julien               Portela   
4  354605  ur.01011577376.25                 Julien               Portela   

      researchers.research_orgs research_org_countries.id  \
0  [grid.11136.34, grid.4444.0]                        FR   
1  [grid.11136.34, grid.4444.0]                        FR   
2  [grid.11136.34, grid.4444.0]                        FR   
3  [grid.11136.34, grid.4444.0]                        FR   
4  [grid.11136.34, grid.4444.0]                        FR   

  research_org_countries.name category_for.id               category_for.name  \
0                      France            2211  11 Medical and Health Sciences   
1                      France            2211  11 Medical and Health Sciences   
2                      France            2211  11 Medical and Health Sciences   
3                      France            2211  11 Medical and Health Sciences   
4                      France            2211  11 Medical and Health Sciences   

   research_org_cities.id  ... funders.id funders.name funders.acronym  \
0                 2987914  ...        NaN          NaN             NaN   
1                 2987914  ...        NaN          NaN             NaN   
2                 2988507  ...        NaN          NaN             NaN   
3                 2988507  ...        NaN          NaN             NaN   
4                 2972315  ...        NaN          NaN             NaN   

  funders.linkout funders.country_name funders.state_name funders.city_name  \
0             NaN                  NaN                NaN               NaN   
1             NaN                  NaN                NaN               NaN   
2             NaN                  NaN                NaN               NaN   
3             NaN                  NaN                NaN               NaN   
4             NaN                  NaN                NaN               NaN   

  funders.latitude funders.longitude funders.types  
0              NaN               NaN           NaN  
1              NaN               NaN           NaN  
2              NaN               NaN           NaN  
3              NaN               NaN           NaN  
4              NaN               NaN           NaN  

[5 rows x 40 columns]


====
 search datasets where funder_countries is not empty return datasets[id+unnest(category_hrcs_rac)+unnest(research_org_states)+unnest(research_org_cities)+unnest(category_bra)+unnest(category_sdg)+unnest(category_hra)+unnest(category_rcdc)+unnest(category_icrp_cso)+unnest(category_hrcs_hc)+unnest(research_org_countries)+unnest(researchers)+unnest(funders)+unnest(category_icrp_ct)+unnest(category_for)+unnest(funder_countries)] 
====

Returned Datasets: 335222 (total = 673883)


id category_sdg.id             category_sdg.name  \
0  354171           40003  3 Good Health and Well Being   
1  354171           40003  3 Good Health and Well Being   
2  354171           40003  3 Good Health and Well Being   
3  354171           40003  3 Good Health and Well Being   
4  354171           40003  3 Good Health and Well Being   

   research_org_cities.id research_org_cities.name     funders.id  \
0                 2220957                  Yaoundé  grid.416870.c   
1                 2220957                  Yaoundé  grid.416870.c   
2                 2220957                  Yaoundé  grid.416870.c   
3                 2220957                  Yaoundé  grid.416870.c   
4                 2220957                  Yaoundé  grid.416870.c   

                                        funders.name funders.acronym  \
0  National Institute of Neurological Disorders a...           NINDS   
1  National Institute of Neurological Disorders a...           NINDS   
2  National Institute of Neurological Disorders a...           NINDS   
3  National Institute of Neurological Disorders a...           NINDS   
4  National Institute of Neurological Disorders a...           NINDS   

               funders.linkout funders.country_name  ... category_for.id  \
0  [http://www.ninds.nih.gov/]        United States  ...             NaN   
1  [http://www.ninds.nih.gov/]        United States  ...             NaN   
2  [http://www.ninds.nih.gov/]        United States  ...             NaN   
3  [http://www.ninds.nih.gov/]        United States  ...             NaN   
4  [http://www.ninds.nih.gov/]        United States  ...             NaN   

  category_for.name  category_rcdc.id  category_rcdc.name category_hrcs_hc.id  \
0               NaN               NaN                 NaN                 NaN   
1               NaN               NaN                 NaN                 NaN   
2               NaN               NaN                 NaN                 NaN   
3               NaN               NaN                 NaN                 NaN   
4               NaN               NaN                 NaN                 NaN   

  category_hrcs_hc.name category_icrp_ct.id category_icrp_ct.name  \
0                   NaN                 NaN                   NaN   
1                   NaN                 NaN                   NaN   
2                   NaN                 NaN                   NaN   
3                   NaN                 NaN                   NaN   
4                   NaN                 NaN                   NaN   

  category_bra.id category_bra.name  
0             NaN               NaN  
1             NaN               NaN  
2             NaN               NaN  
3             NaN               NaN  
4             NaN               NaN  

[5 rows x 36 columns]


====
 search datasets where research_orgs is not empty return datasets[id+unnest(category_hrcs_rac)+unnest(research_org_states)+unnest(research_org_cities)+unnest(category_bra)+unnest(category_sdg)+unnest(category_hra)+unnest(category_rcdc)+unnest(category_icrp_cso)+unnest(category_hrcs_hc)+unnest(research_org_countries)+unnest(researchers)+unnest(funders)+unnest(category_icrp_ct)+unnest(category_for)+unnest(funder_countries)+unnest(research_orgs)] 
====

Returned Datasets: 20623 (total = 1069454)


id research_orgs.id       research_orgs.name  \
0  354641    grid.11136.34  University of Perpignan   
1  354641    grid.11136.34  University of Perpignan   
2  354641    grid.11136.34  University of Perpignan   
3  354641    grid.11136.34  University of Perpignan   
4  354641    grid.11136.34  University of Perpignan   

        research_orgs.linkout research_orgs.country_name  \
0  [http://www.univ-perp.fr/]                     France   
1  [http://www.univ-perp.fr/]                     France   
2  [http://www.univ-perp.fr/]                     France   
3  [http://www.univ-perp.fr/]                     France   
4  [http://www.univ-perp.fr/]                     France   

  research_orgs.state_name research_orgs.city_name  research_orgs.latitude  \
0     Languedoc-Roussillon               Perpignan                 42.6833   
1     Languedoc-Roussillon               Perpignan                 42.6833   
2     Languedoc-Roussillon               Perpignan                 42.6833   
3     Languedoc-Roussillon               Perpignan                 42.6833   
4     Languedoc-Roussillon               Perpignan                 42.6833   

   research_orgs.longitude research_orgs.types  ... category_icrp_ct.id  \
0                  2.90167         [Education]  ...                 NaN   
1                  2.90167         [Education]  ...                 NaN   
2                  2.90167         [Education]  ...                 NaN   
3                  2.90167         [Education]  ...                 NaN   
4                  2.90167         [Education]  ...                 NaN   

  category_icrp_ct.name  category_hrcs_rac.id category_hrcs_rac.name  \
0                   NaN                   NaN                    NaN   
1                   NaN                   NaN                    NaN   
2                   NaN                   NaN                    NaN   
3                   NaN                   NaN                    NaN   
4                   NaN                   NaN                    NaN   

  category_hra.id category_hra.name category_bra.id category_bra.name  \
0             NaN               NaN             NaN               NaN   
1             NaN               NaN             NaN               NaN   
2             NaN               NaN             NaN               NaN   
3             NaN               NaN             NaN               NaN   
4             NaN               NaN             NaN               NaN   

  category_hrcs_hc.id category_hrcs_hc.name  
0                 NaN                   NaN  
1                 NaN                   NaN  
2                 NaN                   NaN  
3                 NaN                   NaN  
4                 NaN                   NaN  

[5 rows x 52 columns]